In [1]:
import os, shutil, re
import pandas as pd
import numpy as np
from glob import glob
from IPython.display import HTML

In [2]:
def get_dirs_at_depth(path, depth_selected, topdown = False):
    res = []
    for root,dirs,files in os.walk(path, topdown=True):
        depth = root[len(path) + len(os.path.sep)-1:].count(os.path.sep)
        if depth == depth_selected:
            res += [os.path.join(root, d) for d in dirs]
            dirs[:] = [] # Don't recurse any deeper
    return res

def copytree(src, dst, symlinks=False, ignore=None):
    if not os.path.exists(dst):
        os.makedirs(dst)
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            copytree(s, d, symlinks, ignore)
        else:
            if not os.path.exists(d) or os.stat(s).st_mtime - os.stat(d).st_mtime > 1:
                shutil.copy2(s, d)

In [3]:
path = '/media/sanromd/data/lab/data/characterization'
outpath = '/media/sanromd/data/lab/data/_characterization_by_sample'

# get dirs
devices = get_dirs_at_depth(path, 0, topdown = True)
meas_tags = get_dirs_at_depth(path, 1)
meas_tags.sort()

#check the outpath exists, if not create it
if not os.path.exists(outpath):
    os.mkdir(outpath)

In [4]:
sample_tag = []
device_tag = []
for meas in meas_tags:
    meas = meas.split(os.path.sep)
    sample_tag += [meas[-1]]
    device_tag += [meas[-2]]
    #res += [device_tag, sample_tag]

In [5]:
df = pd.DataFrame({'device_tag':device_tag, 'sample_tag':sample_tag})

In [6]:
tags = df.loc[:,'sample_tag'].unique()
devs = df.loc[:,'device_tag'].unique()

tags.sort()
devs.sort()

# remove unwanted tags
revised_tags = []
for tag in tags:
     if re.match('[a-z]+[_]+[0-9][0-9][0-9]', tag) is not None:
            revised_tags += [tag]

tags = revised_tags

df2 = pd.DataFrame(index = range(len(tags)), columns = np.append('sample', devs))

for k,tag in enumerate(tags):
        df2.loc[k,'sample_tag'] = tag

df2['sample_code'] = df2.loc[:, 'sample_tag'].apply(lambda x: x.split('_')[0])

df2 = df2.loc[:, np.append(['sample_code', 'sample_tag'], devs)]
df3 = df2.copy()
uvs_path    = os.path.join(path,'uvvisnir','_generic')
uvs_generic = os.listdir(uvs_path)
uvs_generic.sort()
uvs_generic = uvs_generic[2:]

for c in df2['sample_code'].unique():
    if c in uvs_generic:
        df2.loc[df2['sample_code']==c,'uvvisnir'] = '<a href="./{0}/{1}">generic</a>'.format(uvs_path,c)
        df3.loc[df2['sample_code']==c,'uvvisnir'] = os.path.join(uvs_path,c)
uvs_path    = os.path.join(path,'tg_dta','_generic')
uvs_generic = os.listdir(uvs_path)
uvs_generic.sort()
uvs_generic = uvs_generic

for c in df2['sample_code'].unique():
    if c in uvs_generic:
        df2.loc[df2['sample_code']==c,'tg_dta'] = '<a href="./{0}/{1}">generic</a>'.format(uvs_path,c)
        df3.loc[df2['sample_code']==c,'tg_dta'] = os.path.join(uvs_path,c)

for k,tag in enumerate(tags):
        dev_meas = df.loc[df['sample_tag']==tag,'device_tag'].values
        for dev in dev_meas:
            tag_path = tag
            if dev == 'autoscan':
                tag_path = tag+'/_postprocessed'
            df2.loc[k,dev] = '<a href="./{0}/{1}">link</a>'.format(dev,tag_path)#os.path.join(dev,tag)
            df3.loc[k,dev] = os.path.join(dev,tag_path)

df2.replace(np.nan, '-', inplace=True)

In [7]:
df2.to_html(os.path.join(path,'characterization.html'), na_rep='-', escape=False)

In [8]:
for s in df3.iterrows():
    v = s[-1]
    sample_path = os.path.join(outpath,v['sample_tag'])
    if not os.path.exists(sample_path): os.mkdir(sample_path)
    devs = v.iloc[2:].dropna().reset_index()
    for k in range(devs.shape[0]):
        dev = devs.iloc[k,0]
        dev_path = os.path.join(sample_path,dev)
        if not os.path.exists(dev_path): os.mkdir(dev_path)
        src_path = os.path.join(path,devs.iloc[k,1])
        print(dev, dev_path, src_path, sep = '\n')
        copytree(src_path, dev_path)
    #print(v['sample_tag'])

autoscan
/media/sanromd/data/lab/data/_characterization_by_sample/ah_001/autoscan
/media/sanromd/data/lab/data/characterization/autoscan/ah_001/_postprocessed
photo
/media/sanromd/data/lab/data/_characterization_by_sample/ah_001/photo
/media/sanromd/data/lab/data/characterization/photo/ah_001
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/ah_001/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/ah_001
autoscan
/media/sanromd/data/lab/data/_characterization_by_sample/bg_000/autoscan
/media/sanromd/data/lab/data/characterization/autoscan/bg_000/_postprocessed
tg_dta
/media/sanromd/data/lab/data/_characterization_by_sample/bg_000/tg_dta
/media/sanromd/data/lab/data/characterization/tg_dta/_generic/bg
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/bg_000/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/bg
photo
/media/sanromd/data/lab/data/_characterization_by_sample/bg_001/photo
/media/sanromd/data/lab/data/charac

ct_scan
/media/sanromd/data/lab/data/_characterization_by_sample/cb_004/ct_scan
/media/sanromd/data/lab/data/characterization/ct_scan/cb_004
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/cb_004/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/cb
autoscan
/media/sanromd/data/lab/data/_characterization_by_sample/cb_005/autoscan
/media/sanromd/data/lab/data/characterization/autoscan/cb_005/_postprocessed
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/cb_005/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/cb
ct_scan
/media/sanromd/data/lab/data/_characterization_by_sample/cb_006/ct_scan
/media/sanromd/data/lab/data/characterization/ct_scan/cb_006
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/cb_006/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/cb
porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/dle_000/porosometer
/media/sanromd/data/lab

porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/lsp_001/porosometer
/media/sanromd/data/lab/data/characterization/porosometer/lsp_001
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/lsp_001/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/lsp
porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/lsp_002/porosometer
/media/sanromd/data/lab/data/characterization/porosometer/lsp_002
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/lsp_002/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/lsp
porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/lsp_003/porosometer
/media/sanromd/data/lab/data/characterization/porosometer/lsp_003
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/lsp_003/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/lsp
porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/lsp_004/poros

rock_mechanics
/media/sanromd/data/lab/data/_characterization_by_sample/lssy_021/rock_mechanics
/media/sanromd/data/lab/data/characterization/rock_mechanics/lssy_021
tg_dta
/media/sanromd/data/lab/data/_characterization_by_sample/lssy_021/tg_dta
/media/sanromd/data/lab/data/characterization/tg_dta/_generic/lssy
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/lssy_021/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/_generic/lssy
autoscan
/media/sanromd/data/lab/data/_characterization_by_sample/lssy_022/autoscan
/media/sanromd/data/lab/data/characterization/autoscan/lssy_022/_postprocessed
ct_scan
/media/sanromd/data/lab/data/_characterization_by_sample/lssy_022/ct_scan
/media/sanromd/data/lab/data/characterization/ct_scan/lssy_022
rock_mechanics
/media/sanromd/data/lab/data/_characterization_by_sample/lssy_022/rock_mechanics
/media/sanromd/data/lab/data/characterization/rock_mechanics/lssy_022
tg_dta
/media/sanromd/data/lab/data/_characterization_by_sam

porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/lsu_003/porosometer
/media/sanromd/data/lab/data/characterization/porosometer/lsu_003
rock_mechanics
/media/sanromd/data/lab/data/_characterization_by_sample/lsu_003/rock_mechanics
/media/sanromd/data/lab/data/characterization/rock_mechanics/lsu_003
ct_scan
/media/sanromd/data/lab/data/_characterization_by_sample/lsu_004/ct_scan
/media/sanromd/data/lab/data/characterization/ct_scan/lsu_004
porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/lsu_004/porosometer
/media/sanromd/data/lab/data/characterization/porosometer/lsu_004
rock_mechanics
/media/sanromd/data/lab/data/_characterization_by_sample/lsu_004/rock_mechanics
/media/sanromd/data/lab/data/characterization/rock_mechanics/lsu_004
porosometer
/media/sanromd/data/lab/data/_characterization_by_sample/lsy_000/porosometer
/media/sanromd/data/lab/data/characterization/porosometer/lsy_000
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sampl

ct_scan
/media/sanromd/data/lab/data/_characterization_by_sample/sh_003/ct_scan
/media/sanromd/data/lab/data/characterization/ct_scan/sh_003
photo
/media/sanromd/data/lab/data/_characterization_by_sample/sh_003/photo
/media/sanromd/data/lab/data/characterization/photo/sh_003
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/sh_003/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/sh_003
autoscan
/media/sanromd/data/lab/data/_characterization_by_sample/sh_004/autoscan
/media/sanromd/data/lab/data/characterization/autoscan/sh_004/_postprocessed
ct_scan
/media/sanromd/data/lab/data/_characterization_by_sample/sh_004/ct_scan
/media/sanromd/data/lab/data/characterization/ct_scan/sh_004
uvvisnir
/media/sanromd/data/lab/data/_characterization_by_sample/sh_004/uvvisnir
/media/sanromd/data/lab/data/characterization/uvvisnir/sh_004
autoscan
/media/sanromd/data/lab/data/_characterization_by_sample/sh_006/autoscan
/media/sanromd/data/lab/data/characterization/autoscan

In [ ]:
devs.reset_index()

In [ ]:
os.listdir(uvs_path)